In [1]:
import sys
sys.path.append('../')

import os
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from torch.utils.data import  DataLoader

%load_ext autoreload
%autoreload 2
from utils.parser import *
from utils.helper import *
args = parse_args()

/Users/bytedance/miniconda3/envs/counterfactual/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 处理Adult数据集

In [2]:
datafile=args.data_path + args.dataset + '.data'
print(datafile)
if(os.path.exists(datafile)):
    raw_data = np.genfromtxt(datafile,delimiter=', ', dtype=str, invalid_raise=False)
else:
    raw_data = np.genfromtxt('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                        delimiter=', ', dtype=str, invalid_raise=False)
adult_data = pd.DataFrame(raw_data, columns=adult_column_names)
adult_data.head()
# adult_data.info()

../data/adult.data


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
adult_data.drop('fnlwgt', axis = 1, inplace = True)
adult_data.drop('capital-gain', axis = 1, inplace = True)
adult_data.drop('capital-loss', axis = 1, inplace = True)
adult_data.drop('education', axis = 1, inplace = True)
adult_data.drop('relationship', axis = 1, inplace = True)
adult_data.drop('native-country', axis = 1, inplace = True)
adult_data = adult_data.astype({"age": np.int64, "educational-num": np.int64, "hours-per-week": np.int64})
adult_data = adult_data.replace({'workclass': {'Without-pay': 'Other/Unknown', 'Never-worked': 'Other/Unknown'}})
adult_data = adult_data.replace({'workclass': {'Federal-gov': 'Government', 'State-gov': 'Government',
                                    'Local-gov': 'Government'}})
adult_data = adult_data.replace({'workclass': {'Self-emp-not-inc': 'Self-Employed', 'Self-emp-inc': 'Self-Employed'}})
adult_data = adult_data.replace({'workclass': {'?': 'Other/Unknown'}})
adult_data = adult_data.replace(
        {
            'occupation': {
                'Adm-clerical': 'White-Collar', 'Craft-repair': 'Blue-Collar',
                'Exec-managerial': 'White-Collar', 'Farming-fishing': 'Blue-Collar',
                'Handlers-cleaners': 'Blue-Collar',
                'Machine-op-inspct': 'Blue-Collar', 'Other-service': 'Service',
                'Priv-house-serv': 'Service',
                'Prof-specialty': 'Professional', 'Protective-serv': 'Service',
                'Tech-support': 'Service',
                'Transport-moving': 'Blue-Collar', 'Unknown': 'Other/Unknown',
                'Armed-Forces': 'Other/Unknown', '?': 'Other/Unknown'
            }
        }
    )
adult_data = adult_data.replace({'marital-status': {'Married-civ-spouse': 'Married', 'Married-AF-spouse': 'Married',
                                                        'Married-spouse-absent': 'Married', 'Never-married': 'Single'}})
adult_data = adult_data.replace({'income': {'<=50K': 0, '>50K': 1}})
adult_data.head()
# adult_data.info()
# adult_data.apply(lambda x : np.sum(x == " ?"))

,age,workclass,educational-num,marital-status,occupation,race,gender,hours-per-week,income
0,39,Government,13,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,13,Married,White-Collar,White,Male,13,0
2,38,Private,9,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,7,Married,Blue-Collar,Black,Male,40,0
4,28,Private,13,Married,Professional,Black,Female,40,0


In [4]:
df_object_col = [col for col in adult_data.columns if adult_data[col].dtype.name == 'object']
df_int_col = [col for col in adult_data.columns if adult_data[col].dtype.name != 'object' and col != 'income']
target = adult_data["income"]
dataset = pd.concat([adult_data[df_int_col], pd.get_dummies(adult_data[df_object_col]),target], axis = 1)
dataset.head()

,age,educational-num,hours-per-week,workclass_Government,workclass_Other/Unknown,workclass_Private,workclass_Self-Employed,marital-status_Divorced,marital-status_Married,marital-status_Separated,...,occupation_Service,occupation_White-Collar,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male,income
0,39,13,40,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,50,13,13,0,0,0,1,0,1,0,...,0,1,0,0,0,0,1,0,1,0
2,38,9,40,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,53,7,40,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,28,13,40,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0


In [5]:
 # 划分数据集
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)

In [6]:
train_dataset = train_dataset.apply(lambda x : (x - x.mean()) / x.std())
test_dataset = test_dataset.apply(lambda x : (x - x.mean()) / x.std())
train_target, test_target = np.array(y_train), np.array(y_test)
train_dataset, test_dataset = np.array(train_dataset, dtype = np.float32), np.array(test_dataset, dtype = np.float32)
print(train_dataset.shape,test_dataset.shape)

train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle = True, drop_last = False)
test_loader = DataLoader(test_dataset, batch_size = args.batch_size, shuffle = False, drop_last = False)


(26048, 26) (6513, 26)
